In [3]:
import requests, json, lxml
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from time import perf_counter
import pandas as pd

In [36]:
url = 'https://www.immoweb.be/en/classified/mansion/for-sale/puurs/2870/10566398'
content = requests.get(url).content
all_tables = pd.read_html(content,keep_default_na=False)
data_frame_houses = pd.concat(all_tables,  ignore_index=True)

In [ ]:
data_frame_houses

In [38]:
house_dict = {}
# bu sözlüğe önce id eklenebilir 
for i in data_frame_houses.index:
    house_dict[data_frame_houses[0][i]] = data_frame_houses[1][i]
house_dict

{'Construction year': '1909',
 'Building condition': 'To be done up',
 'Street frontage width': '10 m',
 'Number of frontages': '2',
 'Covered parking spaces': '1',
 'Living area': '282  m² square meters',
 'Living room surface': '52  m² square meters',
 'Kitchen type': 'Installed',
 'Kitchen surface': '14  m² square meters',
 'Bedrooms': '4',
 'Bedroom 1 surface': '19  m² square meters',
 'Bedroom 2 surface': '17  m² square meters',
 'Bedroom 3 surface': '17  m² square meters',
 'Bathrooms': '1',
 'Toilets': '2',
 'Office surface': '28  m² square meters',
 'Office': 'Yes',
 'Basement': 'Yes',
 'Surface of the plot': '497  m²  square meters',
 'Width of the lot on the street': '10 m  meters',
 'Gas, water & electricity': 'No',
 'Garden': 'Yes',
 'Garden orientation': 'East',
 'Terrace surface': '40  m²  square meters',
 'Primary energy consumption': '240  kWh/m²  kilowatt hour per square meters',
 'Energy class': 'C',
 '': '',
 'Reference number of the EPC report': '20230308-0002314212

In [20]:
# house_keys = data_frame_houses.iloc[:,0]
# house_values = data_frame_houses.iloc[:,1]

In [39]:
headers_df = [
        "Id",
        "Locality",
        "Type of property",
        "Subtype of property",
        "Price",
        "Type of sale",
        "Number of rooms",
        "Living Area",
        "Fully equipped kitchen",
        "Furnished",
        "Open fire",
        "Terrace",
        "Terrace area",
        "Garden",
        "Garden area",
        "Surface of the land",
        "Surface area of the plot of land",
        "Number of facades",
        "Swimming pool",
        "State of the building",
        "Url"
    ]

df = pd.DataFrame(columns=headers_df)

In [40]:
df

,Locality,Type of property,Subtype of property,Price,Type of sale,Number of rooms,Living Area,Fully equipped kitchen,Furnished,Open fire,...,Terrace area,Garden,Garden area,Surface of the land,Surface area of the plot of land,Number of facades,Swimming pool,State of the building,Id,Url


In [1]:
import requests, json, lxml, re, csv
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from time import perf_counter
import pandas as pd
from typing import List, Dict

class Immoweb():
    root_url = "https://www.immoweb.be/en/search/"
    all_data_file = "all_data.csv"
    search_links = []
    estate_types = ['house', 'apartment']
    all_immo_links = []

    def __init__(self):
        self.search_links = self.get_search_list()
        self.immo_links = self.get_link_from_search_page()
        
    def get_search_list(self) -> List:
        for estate in self.estate_types:
            for page in range(1,3): #for testing purpose I changed from 334 to 2
                url = f"{self.root_url}{estate}/for-sale?countries=BE&page={page}&orderBy=relevance"
                self.search_links.append(url)
        return self.search_links

    def get_link_from_search_page(self):
        with ThreadPoolExecutor() as pool:
            for link in self.search_links:
                req = requests.get(link)
                soup = BeautifulSoup(req.content, 'html.parser')
                card_results = soup.find_all('article', class_='card--result')
                immo_links = []
                for article in card_results:
                    link = article.find('a', class_='card__title-link')
                    if link:
                        immo_links.append(link['href'])
            # return immo_links

    def new(self):
        with ThreadPoolExecutor() as pool:
            self.all_immo_links = list(pool.map(self.get_link_from_search_page(), self.search_links))
        return self.all_immo_links

if __name__ == "__main__":
    main = Immoweb()
    start = perf_counter()
    print(main.new())
    end=perf_counter()
    pool_time = end-start
    print("pool_time: ", pool_time)

TypeError: 'NoneType' object is not callable